In [1]:
"""!conda install -c conda-forge openexr-python -y
!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir 
!pip install --upgrade google-cloud-storage"""

'!conda install -c conda-forge openexr-python -y\n!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir \n!pip install --upgrade google-cloud-storage'

In [38]:
# Imports
import os
import tarfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm

from waymo_open_dataset.wdl_limited.sim_agents_metrics import metric_features
from waymo_open_dataset.wdl_limited.sim_agents_metrics import metrics

from waymo_open_dataset.protos import scenario_pb2
from waymo_open_dataset.protos import sim_agents_metrics_pb2
from waymo_open_dataset.protos import sim_agents_submission_pb2
from google.protobuf import text_format

from waymo_open_dataset.utils.sim_agents import submission_specs
from waymo_open_dataset.utils.sim_agents import test_utils as sim_agents_test_utils
from waymo_open_dataset.utils.sim_agents import visualizations
from waymo_open_dataset.utils import trajectory_utils

# Set matplotlib to jshtml so animations work with colab.
from matplotlib import rc
rc('animation', html='jshtml')

# deactivating the warnings
import warnings
warnings.filterwarnings('ignore')

from download import download_from_gcs
from prepare_dataset import Prepare_train_dataset, Prepare_validation_dataset, Prepare_test_dataset
from data_preprocess import process_waymo_data_with_scenario_proto


# Downloader

In [3]:
# Download samples
download_from_gcs('uncompressed/scenario/validation/validation.tfrecord-00000-of-00150')
download_from_gcs('uncompressed/scenario/training/training.tfrecord-00000-of-01000')
download_from_gcs('uncompressed/scenario/testing/testing.tfrecord-00000-of-00150')

validation.tfrecord-00000-of-00150 already exists in waymo_open_dataset_/validation.tfrecord-00000-of-00150
training.tfrecord-00000-of-01000 already exists in waymo_open_dataset_/training.tfrecord-00000-of-01000
testing.tfrecord-00000-of-00150 already exists in waymo_open_dataset_/testing.tfrecord-00000-of-00150


# Loader

In [9]:
"""# scenario exemple
dataset = Prepare_train_dataset()
# iterate over the dataset
for data in dataset:
    scenario = scenario_pb2.Scenario()
    scenario.ParseFromString(data.numpy())
# describe the scenario
print(scenario.description)"""

    
    


'# scenario exemple\ndataset = Prepare_train_dataset()\n# iterate over the dataset\nfor data in dataset:\n    scenario = scenario_pb2.Scenario()\n    scenario.ParseFromString(data.numpy())\n# describe the scenario\nprint(scenario.description)'

In [39]:
"""# scenario exemple
dataset = Prepare_train_dataset()
dataset_iterator = dataset.as_numpy_iterator()
bytes_example = next(dataset_iterator)
scenario = scenario_pb2.Scenario.FromString(bytes_example)
print(f'Checking type: {type(scenario)}')
print(f'Loaded scenario with ID: {scenario.scenario_id}')

logged_trajectories =trajectory_utils.ObjectTrajectories.from_scenario(scenario)
print(f'Original shape of tensors inside trajectories: {logged_trajectories.valid.shape} (n_objects, n_steps)')
sim_agents_ids = tf.convert_to_tensor(submission_specs.get_sim_agent_ids(scenario))
logged_trajectories = logged_trajectories.gather_objects_by_id(sim_agents_ids)
print(f'Shape of tensors inside trajectories after removing objects that should not be simulated: {logged_trajectories.valid.shape} (n_objects, n_steps)')
current_trajectories = logged_trajectories.slice_time(
      start_index=0, end_index=submission_specs.CURRENT_TIME_INDEX + 1)
print(f'Modified shape of tensors after removing future steps: {current_trajectories.valid.shape} (n_objects, n_steps)')
future_trajectories = logged_trajectories.slice_time(
    start_index=submission_specs.CURRENT_TIME_INDEX + 1, end_index=None)
print(f'Modified shape of tensors inside future trajectories: {future_trajectories.valid.shape} (n_objects, n_steps)')

print(f'Are all agents valid: {tf.reduce_all(current_trajectories.valid[:, -1]).numpy()}')
"""

"logged_trajectories =trajectory_utils.ObjectTrajectories.from_scenario(scenario)\nprint(f'Original shape of tensors inside trajectories: {logged_trajectories.valid.shape} (n_objects, n_steps)')\nsim_agents_ids = tf.convert_to_tensor(submission_specs.get_sim_agent_ids(scenario))\nlogged_trajectories = logged_trajectories.gather_objects_by_id(sim_agents_ids)\nprint(f'Shape of tensors inside trajectories after removing objects that should not be simulated: {logged_trajectories.valid.shape} (n_objects, n_steps)')\ncurrent_trajectories = logged_trajectories.slice_time(\n      start_index=0, end_index=submission_specs.CURRENT_TIME_INDEX + 1)\nprint(f'Modified shape of tensors after removing future steps: {current_trajectories.valid.shape} (n_objects, n_steps)')\nfuture_trajectories = logged_trajectories.slice_time(\n    start_index=submission_specs.CURRENT_TIME_INDEX + 1, end_index=None)\nprint(f'Modified shape of tensors inside future trajectories: {future_trajectories.valid.shape} (n_obje

In [5]:
data_file = 'waymo_open_dataset_/training.tfrecord-00000-of-01000'
output_path = 'scenarios/training'

processed = process_waymo_data_with_scenario_proto(data_file, output_path)"""

"data_file = 'waymo_open_dataset_/training.tfrecord-00000-of-01000'\noutput_path = 'scenarios/training'\n\nprocessed = process_waymo_data_with_scenario_proto(data_file, output_path)"